In [1]:
import pytesseract
from langchain.prompts import PromptTemplate
import pandas as pd
import glob
import openai
import os 

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import VectorDBQA
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

import pandas as pd
from pathlib import Path
from pdf2image import convert_from_path
import pdf2image
import time
import numpy as np
import mimetypes

In [2]:
openai.organization = ###
openai.api_key = ##
os.environ["OPENAI_API_KEY"] = ###

In [3]:
def pdf_to_img(pdf_file):
    return pdf2image.convert_from_path(pdf_file)

In [4]:
breach_parsing_folder_path = '/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/'
breach_april_list = []
for filename in os.scandir(breach_parsing_folder_path):
    if filename.is_file():
        # print(filename.name)
        # print(filename.path)
        breach_april_list.append([filename.path, filename.name])

In [8]:
#temp_file = '/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29402Apr.pdf'
breach_april_list


[['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29384Apr.pdf',
  'letter29384Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29388Apr.pdf',
  'letter29388Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29396Apr.pdf',
  'letter29396Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29410Apr.pdf',
  'letter29410Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29402Apr.pdf',
  'letter29402Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29330Apr.pdf',
  'letter29330Apr.pdf'],
 ['/Users/kassandramadulka/Documents/Projects/breach_parsing/drive-download-20230428T041547Z-001/letter29363Apr_0.pdf',
  'letter29363Apr_0.pd

In [5]:
import time
def get_breach_info(query, folder_path=True, folder_list=False):
    ans_list = []
    pdf_name_list = []
    if folder_list==False:
        for filename in os.scandir(folder_path):
            if filename.is_file():
                pdf_name_list.append(filename.name)
                # get ocr text
                ocrText = ""
                img = pdf_to_img(filename.path)
                for index, page in enumerate(img):
                    ocrText += pytesseract.image_to_string(page)

                ocrText_new = ocrText.replace("Experian", "")
                #ocrText_new = ocrText_new.replace("TransUnion", " ")
                #ocrText_new = ocrText_new.replace("Equifax", " ")
                # remove file first
                #dbutils.fs.rm('dbfs:/FileStore/kmadulka/breach_parsing/txt_output/ocrText.txt', True)
                # save text as txt file:
                with open('/Users/kassandramadulka/Documents/Projects/breach_parsing/ocrText.txt', 'w') as f:
                    f.write(ocrText_new)


                # query the document
                loader = TextLoader('/Users/kassandramadulka/Documents/Projects/breach_parsing/ocrText.txt')
                documents = loader.load()

                # split into chunks if its too big
                text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
                texts = text_splitter.split_documents(documents)

                # define AI model for embeddings
                embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
                vectordb = Chroma.from_documents(texts, embeddings, openai_api_key=openai.api_key)


                # QA
                qa =RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())

                # define prompt template
                prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

                  Instruction: 
                  You are a cyber security auditor and your job is to answer questions based on text from a cybersecurity databreach document. You know that Experian, TransUnion and Equifax are companies that offer credit monitoring services that are also often included in the breach documents even though they are not relevant to the breach event. Therefore, you know to ignore the parts of the text that mention these services.
                  Use only information in the following text to answer the question at the end. \
                  Explain the answer with reference to these text. If you don't know, say 'Unknown'.

                  {context}

                  Question: {question}

                  Response:
                  """
                PROMPT = PromptTemplate(
                    template=prompt_template, input_variables=["context", "question"]
                )

                chain_type_kwargs = {"prompt": PROMPT}
                qa = RetrievalQA.from_chain_type(
                    llm=OpenAI(), chain_type="stuff", retriever=vectordb.as_retriever(), chain_type_kwargs=chain_type_kwargs)

                # query
                #query = "What company was breached? Answer as succinctly as possible. Do not answer in a full sentence."
                ans = qa.run(query)
                print(filename.name)
                print(ans)
                ans_list.append(ans)

                time.sleep(3)
    else:

        for file in folder_list:
            pdf_name_list.append(file[1])
           # get ocr text
            ocrText = ""
            img = pdf_to_img(file[0])
            for index, page in enumerate(img):
              ocrText += pytesseract.image_to_string(page)

            ocrText_new = ocrText.replace("Experian", "")
            #ocrText_new = ocrText_new.replace("TransUnion", " ")
            #ocrText_new = ocrText_new.replace("Equifax", " ")
            # remove file first
            #dbutils.fs.rm('dbfs:/FileStore/kmadulka/breach_parsing/txt_output/ocrText.txt', True)
            # save text as txt file:
            with open('/Users/kassandramadulka/Documents/Projects/breach_parsing/ocrText.txt', 'w') as f:
                f.write(ocrText_new)


            # query the document
            loader = TextLoader('/Users/kassandramadulka/Documents/Projects/breach_parsing/ocrText.txt')
            documents = loader.load()

            # split into chunks if its too big
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
            texts = text_splitter.split_documents(documents)

            # define AI model for embeddings
            embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
            vectordb = Chroma.from_documents(texts, embeddings, openai_api_key=openai.api_key)


            # QA
            qa =RetrievalQA.from_chain_type(llm=ChatOpenAI(model_name="gpt-3.5-turbo"), chain_type="stuff", retriever=vectordb.as_retriever())

            # define prompt template
            prompt_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

              Instruction: 
              You are a cyber security auditor and your job is to answer questions based on text from a cybersecurity databreach document. You know that Experian, TransUnion and Equifax are companies that offer credit monitoring services that are also often included in the breach documents even though they are not relevant to the breach event. Therefore, you know to ignore the parts of the text that mention these services.
              Use only information in the following text to answer the question at the end. \
              Explain the answer with reference to these text. If you don't know, say 'Unknown'.

              {context}

              Question: {question}

              Response:
              """
            PROMPT = PromptTemplate(
                template=prompt_template, input_variables=["context", "question"]
            )

            chain_type_kwargs = {"prompt": PROMPT}
            qa = RetrievalQA.from_chain_type(
                llm=OpenAI(), chain_type="stuff", retriever=vectordb.as_retriever(), chain_type_kwargs=chain_type_kwargs)

            # query
            #query = "What company was breached? Answer as succinctly as possible. Do not answer in a full sentence."
            ans = qa.run(query)
            print(file[1])
            print(ans)
            ans_list.append(ans)

            time.sleep(3) 

    return pdf_name_list, ans_list



In [32]:
query= """What is the name of the company that was breached 
or that was investigated due to a cybersecurity incident according to the text? 
Answer as succinctly as possible. Do not answer in a full sentence. 
If you cannot find the name of the company that was breached 
but you know the name of the company that created the report, 
please provide that company's name instead. If you do not know the name of the company that was breached 
or the name of the company that created the document, answer with 'Unknown'. """

pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[20:30])

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


letter29375Apr.pdf
 90 Degree Benefits-Wisconsin
letter29379Apr.pdf
 Retina & Vitreous of Texas, PLLC
letter29428Apr.pdf
 NationsBenefits
letter29188AprAdditional.pdf
 Woodhaven
letter29367Apr.pdf
 Baldor Specialty Foods
Letter29436Apr.pdf
 MutualOne Bank
letter29394Apr.pdf
 kW Mission Critical Engineering
letter29386Apr.pdf
 Rockland Trust Company
letter29398Apr.pdf

J.P Morgan
letter29432Apr.pdf
 Taro Pharmaceuticals U.S.A., Inc.


In [7]:
labels = pd.read_json('/Users/kassandramadulka/Downloads/breach-data-april-2023 (1).jsonl', lines=True)[['filename', 'organization']]
labels.head()

,filename,organization
0,letter29418Apr.pdf,Nonstop Administration and Insurance Services
1,letter26750AdditionalApril.pdf,"Shields Health Care Group, Inc. (Additional In..."
2,letterAdditional28623Apr.pdf,The Father's Table LLC (Additional Information)
3,letter28627AdditionalApril.pdf,First National Bank of Pennsylvania (Additiona...
4,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ..."


In [12]:
def results_to_df(pdf_name_list, ans_list, col_name='AI_org_name'):
    d = {'filename': pdf_name_list, col_name: ans_list}
    df = pd.DataFrame(data=d)
    
    return df

In [33]:
d = {'filename': pdf_name_list, 'AI_org_name': ans_list}
df = pd.DataFrame(data=d)

final_df = pd.merge(labels, df, on='filename')
final_df.head(11)

,filename,organization,AI_org_name
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven
1,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods
2,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin
3,letter29379Apr.pdf,"Retina & Vitreous of Texas, PLLC","Retina & Vitreous of Texas, PLLC"
4,letter29386Apr.pdf,Rockland Trust,Rockland Trust Company
5,letter29394Apr.pdf,kW Mission Critical Engineering,kW Mission Critical Engineering
6,letter29398Apr.pdf,"JPMorgan Chase Bank, N.A.",\nJ.P Morgan
7,letter29428Apr.pdf,"NationsBenefits Holdings, LLC",NationsBenefits
8,letter29432Apr.pdf,"Taro Pharmaceuticals U.S.A., Inc.","Taro Pharmaceuticals U.S.A., Inc."
9,Letter29436Apr.pdf,Mutual One Bank,MutualOne Bank


In [34]:
query = """What date did the breach happen? Do not respond in a full sentence. 
Please resond with the Month, day, year. Remove any '\n' from the response. 
If you do not know the date of the breach, please respond with 'Unknown'. 
Please convert the date to the format MM/DD/YYYY in your response"""
pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[20:30])
date_df = results_to_df(pdf_name_list, ans_list, col_name='breach_date')
date_df['breach_date'] = date_df['breach_date'].str.replace('\n','')
final_df = pd.merge(final_df, date_df, on='filename')
final_df

letter29375Apr.pdf
 12/05/2022 - 12/11/2022
letter29379Apr.pdf
 02/01/2023
letter29428Apr.pdf
 01/30/2023
letter29188AprAdditional.pdf
 11/16/2022
letter29367Apr.pdf
 02/25/2023
Letter29436Apr.pdf
 Unknown
letter29394Apr.pdf
 Unknown
letter29386Apr.pdf
 04/06/2023
letter29398Apr.pdf

April 13, 2023
letter29432Apr.pdf

04/19/2023


,filename,organization,AI_org_name,breach_date
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven,11/16/2022
1,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods,02/25/2023
2,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin,12/05/2022 - 12/11/2022
3,letter29379Apr.pdf,"Retina & Vitreous of Texas, PLLC","Retina & Vitreous of Texas, PLLC",02/01/2023
4,letter29386Apr.pdf,Rockland Trust,Rockland Trust Company,04/06/2023
5,letter29394Apr.pdf,kW Mission Critical Engineering,kW Mission Critical Engineering,Unknown
6,letter29398Apr.pdf,"JPMorgan Chase Bank, N.A.",\nJ.P Morgan,"April 13, 2023"
7,letter29428Apr.pdf,"NationsBenefits Holdings, LLC",NationsBenefits,01/30/2023
8,letter29432Apr.pdf,"Taro Pharmaceuticals U.S.A., Inc.","Taro Pharmaceuticals U.S.A., Inc.",04/19/2023
9,Letter29436Apr.pdf,Mutual One Bank,MutualOne Bank,Unknown


In [45]:
query = """What date was this document created? 
Do not respond in a full sentence. Please resond with the Month, day, year.
Please convert the date to the format in the format MM/DD/YYYY in your response. 
If you cannot respond with the date in the proper format, please respond with 'Unknown'.
Please convert the date to the format in the format MM/DD/YYYY in your response.
If you do not know the date of the breach, please respond with 'Unknown'."""
pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[20:30])
report_date_df = results_to_df(pdf_name_list, ans_list, col_name='reported_date')
report_date_df['reported_date'] = report_date_df['reported_date'].str.replace('\n','')
final_df = pd.merge(final_df, report_date_df, on='filename')
final_df['breach_date'] = final_df['breach_date'].str.strip()
final_df['reported_date'] = final_df['reported_date'].str.strip()
final_df['breach_date'] = np.where(final_df['breach_date']==final_df['reported_date'], 'Unknown', final_df['breach_date'])
final_df

,filename,organization,AI_org_name,breach_date,reported_date
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven,11/16/2022,03/31/2023
1,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods,02/25/2023,04/07/2023
2,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin,12/05/2022 - 12/11/2022,04/07/2023
3,letter29379Apr.pdf,"Retina & Vitreous of Texas, PLLC","Retina & Vitreous of Texas, PLLC",02/01/2023,04/10/2023
4,letter29386Apr.pdf,Rockland Trust,Rockland Trust Company,04/06/2023,04/13/2023
5,letter29394Apr.pdf,kW Mission Critical Engineering,kW Mission Critical Engineering,Unknown,04/10/2023
6,letter29398Apr.pdf,"JPMorgan Chase Bank, N.A.",\nJ.P Morgan,Unknown,"April 13, 2023"
7,letter29428Apr.pdf,"NationsBenefits Holdings, LLC",NationsBenefits,01/30/2023,Unknown
8,letter29432Apr.pdf,"Taro Pharmaceuticals U.S.A., Inc.","Taro Pharmaceuticals U.S.A., Inc.",Unknown,04/19/2023
9,Letter29436Apr.pdf,Mutual One Bank,MutualOne Bank,Unknown,04/17/2023


In [46]:
query= """Find the company that was breached according to this document. 
Please ignore any information about credit monitoring services from Equifax, Experian, or TransUnion. 
This is irrelvant to the breach incident.
If the company was directly breached or had information leaked directly from a cybersecurity incidient, 
respond with 'first' to indicate a first party breach. 
If the document mentions that the notice is due to a vendor or a partner company that was breached 
then respond with 'third' to indicate a third party breach. 
Do not confuse a company being breached by a third-party with a third party breach.
Responses should only be one word: 'first' or 'third'. Do not eleborate.
"""
# Please explain why you believe it is a first or third party breach. 

pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[20:30])
party_df = results_to_df(pdf_name_list, ans_list, col_name='first_or_third')
party_df['first_or_third'] = party_df['first_or_third'].str.replace('\n','')
final_df = pd.merge(final_df, party_df, on='filename')
final_df

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


letter29375Apr.pdf

first
letter29379Apr.pdf

first
letter29428Apr.pdf
 Third
letter29188AprAdditional.pdf

first
letter29367Apr.pdf

first
Letter29436Apr.pdf
 Unknown
letter29394Apr.pdf
 First
letter29386Apr.pdf
 Unknown.
letter29398Apr.pdf
 First
letter29432Apr.pdf

first


,filename,organization,AI_org_name,breach_date,reported_date,first_or_third
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven,11/16/2022,03/31/2023,first
1,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods,02/25/2023,04/07/2023,first
2,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin,12/05/2022 - 12/11/2022,04/07/2023,first
3,letter29379Apr.pdf,"Retina & Vitreous of Texas, PLLC","Retina & Vitreous of Texas, PLLC",02/01/2023,04/10/2023,first
4,letter29386Apr.pdf,Rockland Trust,Rockland Trust Company,04/06/2023,04/13/2023,Unknown.
5,letter29394Apr.pdf,kW Mission Critical Engineering,kW Mission Critical Engineering,Unknown,04/10/2023,First
6,letter29398Apr.pdf,"JPMorgan Chase Bank, N.A.",\nJ.P Morgan,Unknown,"April 13, 2023",First
7,letter29428Apr.pdf,"NationsBenefits Holdings, LLC",NationsBenefits,01/30/2023,Unknown,Third
8,letter29432Apr.pdf,"Taro Pharmaceuticals U.S.A., Inc.","Taro Pharmaceuticals U.S.A., Inc.",Unknown,04/19/2023,first
9,Letter29436Apr.pdf,Mutual One Bank,MutualOne Bank,Unknown,04/17/2023,Unknown


In [47]:
query = "What type of data or information was stolen in this data breach? Please respond in english."
pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[20:30])
info_df = results_to_df(pdf_name_list, ans_list, col_name='information_leaked')
final_df = pd.merge(final_df, info_df, on='filename')
final_df

letter29375Apr.pdf
 The data or information stolen in this data breach included names, addresses, dates of birth, Social Security numbers, medical/health information, and/or information related to the payment of healthcare services.
letter29379Apr.pdf
 The data stolen in this data breach included names, addresses, dates of birth, diagnosis and treatment information, insurance carrier information, and insurance subscriber identification numbers.
letter29428Apr.pdf

The personal information involved in this data breach included benefits, credit card account statements, and other identity information such as name, address, date of birth, social security number, etc.
letter29188AprAdditional.pdf
 The data or information stolen in this data breach included name and copies of checks.
letter29367Apr.pdf
 The type of data or information that was potentially stolen included personal information such as name, address, date of birth, Social Security number, insurance information and other benefit

,filename,organization,AI_org_name,breach_date,reported_date,first_or_third,information_leaked
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven,11/16/2022,03/31/2023,first,The data or information stolen in this data b...
1,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods,02/25/2023,04/07/2023,first,The type of data or information that was pote...
2,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin,12/05/2022 - 12/11/2022,04/07/2023,first,The data or information stolen in this data b...
3,letter29379Apr.pdf,"Retina & Vitreous of Texas, PLLC","Retina & Vitreous of Texas, PLLC",02/01/2023,04/10/2023,first,The data stolen in this data breach included ...
4,letter29386Apr.pdf,Rockland Trust,Rockland Trust Company,04/06/2023,04/13/2023,Unknown.,\nThe data or information that was stolen in t...
5,letter29394Apr.pdf,kW Mission Critical Engineering,kW Mission Critical Engineering,Unknown,04/10/2023,First,The data breach involved personal information...
6,letter29398Apr.pdf,"JPMorgan Chase Bank, N.A.",\nJ.P Morgan,Unknown,"April 13, 2023",First,\nThe data breach included personal informatio...
7,letter29428Apr.pdf,"NationsBenefits Holdings, LLC",NationsBenefits,01/30/2023,Unknown,Third,\nThe personal information involved in this da...
8,letter29432Apr.pdf,"Taro Pharmaceuticals U.S.A., Inc.","Taro Pharmaceuticals U.S.A., Inc.",Unknown,04/19/2023,first,\nThe data breach involved personal informatio...
9,Letter29436Apr.pdf,Mutual One Bank,MutualOne Bank,Unknown,04/17/2023,Unknown,Unknown. The text does not provide any inform...


In [49]:
query= """What is the name of the company that was breached 
or that was investigated due to a cybersecurity incident according to the text? 
Answer as succinctly as possible. Do not answer in a full sentence. 
If you cannot find the name of the company that was breached 
but you know the name of the company that created the report, 
please provide that company's name instead. If you do not know the name of the company that was breached 
or the name of the company that created the document, answer with 'Unknown'. """

pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[0:30])
d = {'filename': pdf_name_list, 'AI_org_name': ans_list}
df = pd.DataFrame(data=d)

final_df_metric = pd.merge(labels, df, on='filename')
final_df_metric.head(35)

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')).


letter29384Apr.pdf
 Bank of America
letter29388Apr.pdf
 Brigham & Women's Hospital
letter29396Apr.pdf
 Mercy Ships
letter29410Apr.pdf
 Traditions Bank
letter29402Apr.pdf
 Yum!
letter29330Apr.pdf
 Penn Power Group
letter29363Apr_0.pdf
 TIAA Bank
Letter29449Apr.pdf
 MutualOne Bank
letter29365Apr.pdf
 Unknown
letter29426Apr.pdf
 MutualOne Bank
letter29377Apr.pdf
 Unknown.
letter29385Apri.pdf
 Integreon, Inc.
letter29445Apr.pdf
 CIC Group, Inc.
letter29438Apr.pdf
 BISSELL
letter29369Apr.pdf
 MutualOne Bank
letter29351Apr.pdf

Fresh Del Monte
letter29343Apr.pdf
 Hankins & Sohn
letter29412Apr.pdf
 WRI/Kimco
Letter29447Apr.pdf
 MutualOne Bank


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


letter29424Apr.pdf
 Rubino & Company, Chartered
letter29375Apr.pdf
 90 Degree Benefits-Wisconsin
letter29379Apr.pdf
 Retina & Vitreous of Texas, PLLC
letter29428Apr.pdf
 NationsBenefits Holding, LLC
letter29188AprAdditional.pdf
 Woodhaven
letter29367Apr.pdf
 Baldor Specialty Foods
Letter29436Apr.pdf
 MutualOne Bank


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


letter29394Apr.pdf
 kW Mission Critical Engineering (“kW MCE”)
letter29386Apr.pdf
 Rockland Trust Company
letter29398Apr.pdf
 J.P. Morgan
letter29432Apr.pdf
 Taro Pharmaceuticals U.S.A., Inc.


,filename,organization,AI_org_name
0,letter29188AprAdditional.pdf,"Woodhaven Lumber & Millwork, Inc. (Additional ...",Woodhaven
1,letter29330Apr.pdf,Penn Power Group,Penn Power Group
2,letter29343Apr.pdf,Hankins & Sohn Plastic Surgery Associates,Hankins & Sohn
3,letter29351Apr.pdf,Fresh Del Monte Produce Inc.,\nFresh Del Monte
4,letter29363Apr_0.pdf,"TIAA, FSB",TIAA Bank
5,letter29365Apr.pdf,"Armstrong Transfer & Storage Co., Inc. dba The...",Unknown
6,letter29367Apr.pdf,Baldor Specialty Foods,Baldor Specialty Foods
7,letter29369Apr.pdf,Mutual One Bank,MutualOne Bank
8,letter29375Apr.pdf,"90 Degree Benefits, Inc. -Wisconsin",90 Degree Benefits-Wisconsin
9,letter29377Apr.pdf,"Flatirons Solutions, Inc.",Unknown.


In [51]:
# query= """What is the name of the company that is responsible for the breach according to the text? 
# Answer as succinctly as possible. Do not answer in a full sentence. 
# If you do not know the name of the company is responsible for the breach, answer with 'Unknown'. """

# pdf_name_list, ans_list = get_breach_info(query, folder_list=breach_april_list[0:30])
# d = {'filename': pdf_name_list, 'responsible_for_breach': ans_list}
# df = pd.DataFrame(data=d)

## Match Company Name to Domain

In [52]:
! pip install google-search-results

In [102]:
! pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 kB 1.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.2 MB/s eta 0:00:0000:0100:01


In [103]:
from Levenshtein import distance

In [71]:
from urllib.parse import urlparse

def url_parser(url):
    
    parts = urlparse(url)
    directories = parts.path.strip('/').split('/')
    queries = parts.query.strip('&').split('&')
    
    elements = {
        'scheme': parts.scheme,
        'netloc': parts.netloc,
        'path': parts.path,
        'params': parts.params,
        'query': parts.query,
        'fragment': parts.fragment,
        'directories': directories,
        'queries': queries,
    }
    
    return elements

In [55]:
company_name_list = list(final_df['AI_org_name'].values)
company_name_list

[' Woodhaven',
 ' Baldor Specialty Foods',
 ' 90 Degree Benefits-Wisconsin',
 ' Retina & Vitreous of Texas, PLLC',
 ' Rockland Trust Company',
 ' kW Mission Critical Engineering',
 '\nJ.P Morgan',
 ' NationsBenefits',
 ' Taro Pharmaceuticals U.S.A., Inc.',
 ' MutualOne Bank']

In [125]:
def Sort_Tuple(tup):
 
    # getting length of list of tuples
    lst = len(tup)
    for i in range(0, lst):
 
        for j in range(0, lst-i-1):
            if (tup[j][1] > tup[j + 1][1]):
                temp = tup[j]
                tup[j] = tup[j + 1]
                tup[j + 1] = temp
    return tup

def get_domain(company_name):
    company_name = company_name.strip()
    params = {
      "engine": "google",
      "q": "{} website".format(company_name),
      "api_key": "d26afa74c42c443b1fb2b6f346b5059064f42e8ae9d9ebd7ef0c251b1dd71831"
    }

    search = GoogleSearch(params)
    results = search.get_dict()
    organic_results = results["organic_results"]
    
    # ignore_list = ['facebook', 'instagram', 'linkedin', 'indeed', 'twitter', 'glassdoor', 'wikipedia', 'bloomberg', 'dnb', 'jdsupra', 'zoominfo', 'bbb', 'prnewswire']
    ignore_list =['wikipedia']


    final_domain_list = []
    for i in range(0, len(organic_results)):
        if any(word in organic_results[i]['link'] for word in ignore_list):
            pass
        else:
            distance_score = distance(company_name, organic_results[i]['source'])
            final_domain_list.append((url_parser(organic_results[i]['link'])['netloc'].replace('www.', ''), distance_score))
        final_domain_set = list(set(final_domain_list))
        
    
    # get distance between strings
    
                   
    return final_domain_set
    
    

In [127]:
# use multiple sources
# potentially scrape html file
# Mo also worked on this and looked at other information
# Diffbot - create sample account ~ 100
# from Levenshtein import distance



Sort_Tuple(list(get_domain('Woodhaven')))[0][0]

'woodhaven.com'

In [126]:
for company in company_name_list:
    print(company)
    print(Sort_Tuple(get_domain(company)))

 Woodhaven
[('woodhaven.com', 4), ('woodhavencustomcalls.com', 13), ('woodhaven.org', 15), ('woodhavenmanagement.net', 15), ('woodhavenmi.org', 17), ('woodhavenlumber.com', 18), ('rightturndrivingschool.com', 23), ('nces.ed.gov', 26), ('mywbsd.org', 27), ('woodhavenfh.com', 32)]
 Baldor Specialty Foods
[('baldorfood.com', 0), ('facebook.com', 18), ('bloomberg.com', 18), ('rocketreach.co', 19), ('twitter.com', 20), ('dnb.com', 20), ('linkedin.com', 21), ('instagram.com', 21)]
 90 Degree Benefits-Wisconsin
[('portal.90degreebenefits.com', 10), ('90degreebenefits.com', 10), ('markets.businessinsider.com', 23), ('prnewswire.com', 24), ('jdsupra.com', 26)]
 Retina & Vitreous of Texas, PLLC
[('retinatexas.com', 9), ('ophthalmologytimes.com', 27), ('healthgrades.com', 27), ('prnewswire.com', 28), ('crunchbase.com', 28), ('jdsupra.com', 29), ('linkedin.com', 29), ('facebook.com', 29), ('zoominfo.com', 30)]
 Rockland Trust Company
[('rocklandtrust.com', 8), ('rfcu.com', 17), ('linkedin.com', 19

## End Notebook

In [96]:
# ! pip install setuptools==58
# ! pip install diffbot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 816.3/816.3 kB 8.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: setuptools
    Found existing installation: setuptools 56.0.0
    Uninstalling setuptools-56.0.0:
      Successfully uninstalled setuptools-56.0.0
  Using cached diffbot-2.0.0.tar.gz (5.1 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [1 lines of output]
      error in diffbot setup command: use_2to3 is invalid.
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× Getting requirements to build wheel did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [93]:
# import diffbot
# # client = diffbot.Client(token='f06d8f4ee6a77705b795df5a27d9a4a4')
# import requests

# url = 'https://api.diffbot.com/v3/analyze?token=TOKEN&url=URL'

# response = requests.request('GET', url)

# print(response.text)

ModuleNotFoundError: No module named 'diffbot'

In [ ]:
# def get_diffbot_json(token:str, json_link:str):
#     query = json_link.split('query=')[1]
#     return (
#         f"https://kg.diffbot.com/kg/dql_endpoint?token={token}"
#         f"&type=query&query={query}"
#     )
# import requests

# r = requests.get(url)